<h1 style="
  font-size: 55px; 
  text-align: center; 
  font-family: 'Poppins', sans-serif; 
  text-shadow: 2px 2px 5px rgba(63, 155, 77, 0.84);
  letter-spacing: 2px;
">
  <b>📊 Previous Dataset</b>
</h1>
<hr style="width: 60%; border: 2px solid #21996fd1; border-radius: 5px;">


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv("previous_application.csv")

## <span style="color:#21996fd1;">**1- Initial Exploration  & EDA**
---

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

##### <span style="color:#7dc7add1;">**Show Null Percentage in Every Column**

In [ ]:
null_percents = df.isnull().mean() * 100
sorted_nulls = null_percents.sort_values(ascending=False)
max_col_length = max(len(col) for col in df.columns)

print(f"{'Column Name'.ljust(max_col_length)} | Null %")
print("-" * (max_col_length + 10))

for col, percent in sorted_nulls.items():
    print(f"{col.ljust(max_col_length)} | {percent:6.4f}%")

## <span style="color:#21996fd1;"> **2- Data Cleaning**
---

### <span style="color:#7dc7add1;">**=> Drop Columns**

In [ ]:
# Drop 'SK_ID_PREV' not important
df.drop(columns=['SK_ID_PREV'], inplace=True)

In [ ]:
# Drop RATE_INTEREST_PRIVILEGED , RATE_INTEREST_PRIMARY ---> +99% Nulls
df.drop(['RATE_INTEREST_PRIVILEGED', 'RATE_INTEREST_PRIMARY'], axis=1, inplace=True)

In [ ]:
# Drop AMT_DOWN_PAYMENT , RATE_DOWN_PAYMENT ---> 53.6%
df.drop(columns=['AMT_DOWN_PAYMENT','RATE_DOWN_PAYMENT'], inplace=True)

In [ ]:
# Drop NAME_TYPE_SUITE ---> 49% Nulls + Not important in previous data
df.drop('NAME_TYPE_SUITE',axis=1,inplace=True)

In [ ]:
#Show Nulls Percentage & Unique values in ["DAYS_TERMINATION", "DAYS_FIRST_DRAWING", "DAYS_FIRST_DUE", "DAYS_LAST_DUE_1ST_VERSION", "DAYS_LAST_DUE"]
cols = ["DAYS_TERMINATION", "DAYS_FIRST_DRAWING", "DAYS_FIRST_DUE", "DAYS_LAST_DUE_1ST_VERSION", "DAYS_LAST_DUE"]

for col in cols:
    null_count = df[col].isna().sum()
    null_percent = (null_count / len(df)) * 100
    uniq_values = df[col].unique()
    
    print(f"Column : {col}\nNulls Percentage : {null_percent}\nUnique values : {uniq_values}")

In [ ]:
# All of this columns contain about 40.30% Nulls percenatge value
# This is an unreal value in columns as 365243 = placeholder is used instead of missing values ​​(like NULL).It often appears with rejected or canceled loans. 
df.drop(columns=[
    'DAYS_FIRST_DRAWING',
    'DAYS_FIRST_DUE',
    'DAYS_LAST_DUE_1ST_VERSION',
    'DAYS_LAST_DUE',
    'NFLAG_INSURED_ON_APPROVAL',
    'DAYS_TERMINATION'
], inplace=True)

In [ ]:
df.shape

### <span style="color:#7dc7add1;">**=> Handling Nulls**
---

In [ ]:
df.isna().sum()

##### <span style="color:#a8dbc9d1;">Handling AMT_ANNUITY column

In [ ]:
null_percent = df['AMT_ANNUITY'].isnull().sum() / len(df) * 100
print(f"{null_percent:.4f}%")


In [ ]:
df['AMT_ANNUITY'] = df.groupby(['NAME_CONTRACT_TYPE', 'AMT_CREDIT'])['AMT_ANNUITY'].transform(
    lambda x: x.fillna(x.median())
)
df['AMT_ANNUITY'].fillna(df['AMT_ANNUITY'].mean(),inplace=True)

##### <span style="color:#a8dbc9d1;">Handling AMT_GOODS_PRICE column

In [ ]:
null_percent = df['AMT_GOODS_PRICE'].isnull().sum() / len(df) * 100
print(f"missing values: {null_percent:.4f}%")


In [ ]:
df['AMT_GOODS_PRICE'] = df.groupby(['NAME_CONTRACT_TYPE'])['AMT_GOODS_PRICE'].transform(
    lambda x: x.fillna(x.median())
)
df['AMT_GOODS_PRICE'].fillna(df['AMT_GOODS_PRICE'].mean(),inplace=True)

##### <span style="color:#a8dbc9d1;">Handling CNT_PAYMENT Nulls

In [ ]:
null_percent = df['CNT_PAYMENT'].isnull().sum() / len(df) * 100
print(f"{null_percent:.4f}%")

In [ ]:
df['CNT_PAYMENT'] = df.groupby(['NAME_CONTRACT_STATUS', 'NAME_CONTRACT_TYPE'])['CNT_PAYMENT']\
    .transform(lambda x: x.fillna(x.median()))
df['CNT_PAYMENT'] = df['CNT_PAYMENT'].fillna(df['CNT_PAYMENT'].mean())

##### <span style="color:#a8dbc9d1;">Handling AMT_CREDIT Nulls

In [ ]:
xna_percent = df['AMT_CREDIT'].isnull().sum() / len(df) * 100
print(f"{xna_percent:.4f}%")

In [ ]:
df = df.dropna(subset=['AMT_CREDIT'])

xna_percent = df['AMT_CREDIT'].isnull().sum() / len(df) * 100
print(f"{xna_percent:.4f}%")

##### <span style="color:#a8dbc9d1;">Handling PRODUCT_COMBINATION Nulls

In [ ]:
df['PRODUCT_COMBINATION'] = df['PRODUCT_COMBINATION'].fillna(df['PRODUCT_COMBINATION'].mode().iloc[0])

In [ ]:
df.isna().sum()

### <span style="color:#7dc7add1;">**=> Handling XNA & Unknown Data**
---

##### <span style="color:#a8dbc9d1;">**Show XNA Percentage in each column conatin it**

In [ ]:
xna_percent = (df.isin(['XNA']).sum() / len(df)) * 100
xap_percent = (df.isin(['XAP']).sum() / len(df)) * 100

xna_xpa_df = pd.DataFrame({
    'XNA %': xna_percent,
    'XAP %': xap_percent
})

xna_xpa_df = xna_xpa_df[(xna_xpa_df['XNA %'] > 0) | (xna_xpa_df['XAP %'] > 0)]

xna_xpa_df = xna_xpa_df.sort_values(by='XNA %', ascending=False)

max_col_length = max(len(col) for col in xna_xpa_df.index) if not xna_xpa_df.empty else 100
print(f"{'Column Name'.ljust(max_col_length)} | {'XNA %'.rjust(8)} | {'XAP %'.rjust(8)}")
print("-" * (max_col_length + 25))

for col, row in xna_xpa_df.iterrows():
    print(f"{col.ljust(max_col_length)} | {row['XNA %']:8.4f} | {row['XAP %']:8.4f}")


In [ ]:
Columns = ['NAME_CASH_LOAN_PURPOSE','CODE_REJECT_REASON','NAME_GOODS_CATEGORY','NAME_PRODUCT_TYPE','SELLERPLACE_AREA','NAME_SELLER_INDUSTRY']
for col in Columns:
    print(f"Row {col}:\n")
    print(df[col].value_counts())
    print("\n-" * 15,"")

In [ ]:
df.drop(columns=[
    'NAME_CASH_LOAN_PURPOSE', # 1352777 XAP
    'CODE_REJECT_REASON', # 1352777 XNA
    'NAME_GOODS_CATEGORY', # 56% NULLS + 950462 XNA
    'NAME_PRODUCT_TYPE',  # 1063320 XNA
    'SELLERPLACE_AREA', # Not Important + [762672 row = -1]
], inplace=True)

##### <span style="color:#a8dbc9d1;">Handle XNA values in NAME_CONTRACT_TYPE

In [ ]:
df['NAME_CONTRACT_TYPE'].unique()

In [ ]:
df = df[df['NAME_CONTRACT_TYPE'] != 'XNA']

##### <span style="color:#a8dbc9d1;">Handle XNA in NAME_PORTFOLIO

In [ ]:
df['NAME_PORTFOLIO'].unique()

In [ ]:
df['NAME_PORTFOLIO'].value_counts(normalize=True)

In [ ]:
NAME_PORTFOLIO_count = (df['NAME_PORTFOLIO'] == 'XNA').sum()
NAME_PORTFOLIO_percentage = (NAME_PORTFOLIO_count / len(df)) * 100
NAME_PORTFOLIO_percentage

In [ ]:
def fix_portfolio(row):
    current = str(row['NAME_PORTFOLIO']).strip()
    if current != 'XNA':
        return current 

    product = str(row['PRODUCT_COMBINATION']).lower().strip() if pd.notna(row['PRODUCT_COMBINATION']) else ''
    industry = str(row['NAME_SELLER_INDUSTRY']).lower().strip() if pd.notna(row['NAME_SELLER_INDUSTRY']) else ''

    if product.startswith('cash'):
        return 'Cash'
    elif product.startswith('card'):
        return 'Cards'
    elif product.startswith('pos'):
        if 'industry' in industry:
            return 'Cars'
        else:
            return 'POS'
    else:
        return current

df['NAME_PORTFOLIO'] = df.apply(fix_portfolio, axis=1)

In [ ]:
df.drop('NAME_SELLER_INDUSTRY',axis=1,inplace=True) # 51% NULLS + 855716 XNA

In [ ]:
df['NAME_PORTFOLIO'].value_counts(normalize=True)